# Cluster classification with Scikit-learn classifiers

In [1]:
import numpy as np
import pandas as pd
import time
import multiprocessing
import functools
from joblib import Parallel, delayed
import sys

import sklearn
from sklearn import *
from sklearn.experimental import enable_hist_gradient_boosting

sys.path.append("..")
from helpers import filename_for

Load ALL the classification models from scikit-learn.
Note that some models are very slow to train with large datasets or crash outright, so we give them a reduced number of (shuffled) rows to learn.
Note that `n_jobs=1` is used, as parallelism is introduced later.

In [2]:
models_a1 = [
    ('BaggingClassifier', sklearn.ensemble.BaggingClassifier(n_jobs=1), 'medi'),
    ('BernoulliNB', sklearn.naive_bayes.BernoulliNB(), 'fast'),
    ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV(cv=5), 'medi'),
    ('ComplementNB', sklearn.naive_bayes.ComplementNB(), 'fast'),
    ('GaussianNB', sklearn.naive_bayes.GaussianNB(), 'fast'),
]

models_a2 = [
    ('LinearDiscriminantAnalysis', sklearn.discriminant_analysis.LinearDiscriminantAnalysis(), 'fast'),
    ('LinearSVC', sklearn.svm.LinearSVC(max_iter=200000), 'medi'),  # slow with unscaled data
    ('LogisticRegression', sklearn.linear_model.LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=200000), 'medi'),
    ('LogisticRegressionCV', sklearn.linear_model.LogisticRegressionCV(cv=5, solver='lbfgs', multi_class='auto', max_iter=200000), 'medi'),
    ('MLPClassifier', sklearn.neural_network.MLPClassifier(), 'slow'),
    ('MultinomialNB', sklearn.naive_bayes.MultinomialNB(), 'fast'),
]

models_b = [
    ('NearestCentroid', sklearn.neighbors.NearestCentroid(), 'fast'),
    ('PassiveAggressiveClassifier', sklearn.linear_model.PassiveAggressiveClassifier(max_iter=1000, tol=1e-3, n_jobs=1), 'fast'),
    ('Perceptron', sklearn.linear_model.Perceptron(n_jobs=1), 'fast'),
    ('QuadraticDiscriminantAnalysis', sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(), 'fast'),
    ('RidgeClassifier', sklearn.linear_model.RidgeClassifier(), 'fast'),
    ('RidgeClassifierCV', sklearn.linear_model.RidgeClassifierCV(), 'fast'),
    ('SGDClassifier', sklearn.linear_model.SGDClassifier(max_iter=5000, tol=1e-3, n_jobs=1), 'fast'),

]

# Run these sequential due to "buffer source array is read-only" with LokyBackend
models_s = [
    ('AdaBoostClassifier', sklearn.ensemble.AdaBoostClassifier(), 'medi'),
    ('DecisionTreeClassifier', sklearn.tree.DecisionTreeClassifier(), 'fast'),  # medi
    ('ExtraTreeClassifier', sklearn.tree.ExtraTreeClassifier(), 'fast'),  # medi
    ('ExtraTreesClassifier', sklearn.ensemble.ExtraTreesClassifier(n_estimators=100, n_jobs=-1), 'fast'),  # medi
    ('RandomForestClassifier', sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1), 'fast'),  # medi
]

# Models that fail alot
models_o = [
    # ('NuSVC', sklearn.svm.NuSVC(), 'fast'),  # nu infeasible 
    ('RadiusNeighborsClassifier', sklearn.neighbors.RadiusNeighborsClassifier(radius=5, n_jobs=-1, outlier_label=0.), 'medi'),
    # ('GaussianProcessClassifier', sklearn.gaussian_process.GaussianProcessClassifier(), 'slow'),
    # ('GradientBoostingClassifier', sklearn.ensemble.GradientBoostingClassifier(), 'slow'),  # crashes
    # ('HistGradientBoostingClassifier', sklearn.ensemble.HistGradientBoostingClassifier(), 'slow'),  # crashes? 
    ('KNeighborsClassifier', sklearn.neighbors.KNeighborsClassifier(n_jobs=2), 'medi'),  # medi
    # ('LabelPropagation', sklearn.semi_supervised.LabelPropagation(), 'slow'),  # requires too much memory to train
    # ('LabelSpreading', sklearn.semi_supervised.LabelSpreading(), 'slow'),  # requires too much memory to train
    ('SVC', sklearn.svm.SVC(gamma='scale'), 'slow'), # slow, not timeouted 
]

Some models only work with properly scaled data, so we prepare ALL available scalers.

In [3]:
scalers = [
    # ('Unscaled data', ),
    ("standard scaling", sklearn.preprocessing.StandardScaler()),
    ("min-max scaling", sklearn.preprocessing.MinMaxScaler()),
    ("max-abs scaling", sklearn.preprocessing.MaxAbsScaler()),
    ("robust scaling", sklearn.preprocessing.RobustScaler(quantile_range=(25, 75))),
    # ('power transformation (Yeo-Johnson)', sklearn.preprocessing.PowerTransformer(method='yeo-johnson')),
    # ('power transformation (Box-Cox)', sklearn.preprocessing.PowerTransformer(method='box-cox')), # 'strictly zero' meh.
    ("quantile transformation (gaussian pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="normal")),
    ("quantile transformation (uniform pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="uniform")),
    ("sample-wise L2 normalizing", sklearn.preprocessing.Normalizer()),
]

Train and Test data is passed through the scalers, including the "unscaled" scaler.

In [4]:
files = [filename_for(15, 12, 600, 500, n, "inclxx", s, "clusterfeature.pkl") for n in [1, 2, 3, 4] for s in range(20)]
dfs = [pd.read_pickle(file) for file in files]
data = pd.concat(dfs, ignore_index=True).sample(frac=1, random_state=1337)

data = data[(abs(data["X"]) < 125) & (abs(data["Y"]) < 125) & (data["T"] < 100)]

print(data["prim"].value_counts())
display(data)

0.0    4225482
1.0    1412871
Name: prim, dtype: int64


,i_event,prim,T,E,Size,EToF,EnergyMoment,TSpawn,MaxEHit,X,Y,Z
1278499,201.0,0.0,72.743546,4.001890,1.0,356.984039,0.000000e+00,0.000000,4.001890,-7.500000,-9.346230,1507.5
123683,2819.0,0.0,70.245926,4.904631,2.0,445.918732,2.805709e+00,0.121427,2.531206,-6.497627,7.500000,1552.5
5226144,5531.0,1.0,62.695213,320.804871,10.0,625.841431,1.043842e+01,1.434420,94.084465,-17.500000,-4.665872,1507.5
2404152,6755.0,1.0,64.116348,337.163696,13.0,576.190308,1.368203e+01,2.412778,84.405426,-26.984325,2.500000,1512.5
5462128,6398.0,1.0,67.126511,264.252136,9.0,590.024841,8.335863e+00,0.875057,45.102974,12.224815,-12.500000,1592.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1428135,5803.0,0.0,82.358917,1.178269,1.0,295.601349,2.273737e-13,0.000000,1.178269,-22.500000,1.662647,1607.5
480729,6068.0,0.0,78.656456,1.329785,1.0,283.639130,2.009718e-14,0.000000,1.329785,-117.500000,-85.086517,1507.5
3361959,7596.0,0.0,67.661819,41.934479,3.0,556.118164,3.580651e+00,0.262080,19.005957,-24.975262,-27.500000,1582.5
2065244,7478.0,0.0,68.507622,3.715144,1.0,515.199280,5.024296e-15,0.000000,3.715144,-22.107210,17.500000,1572.5


In [5]:
prim1 = data[data["prim"] == 1]
prim0 = data[data["prim"] == 0].sample(n=len(prim1.index), random_state=1337)
balanced_data = pd.concat([prim0, prim1], ignore_index=True).sample(frac=1, random_state=1337)

print(balanced_data["prim"].value_counts())

1.0    1412871
0.0    1412871
Name: prim, dtype: int64


In [6]:
msk = np.random.rand(len(balanced_data)) < 0.8
traindata = balanced_data[msk]
testdata = balanced_data[~msk]

print(traindata.shape, testdata.shape)

(2260653, 12) (565089, 12)


In [7]:
trainscaled = [
    (
        "Unscaled data",
        traindata[["T", "E", "Size", "EToF", "EnergyMoment", "TSpawn", "MaxEHit", "X", "Y", "Z"]],
        testdata[["T", "E", "Size", "EToF", "EnergyMoment", "TSpawn", "MaxEHit", "X", "Y", "Z"]],
    )
] + [
    (
        sname,
        scaler.fit_transform(traindata[["T", "E", "Size", "EToF", "EnergyMoment", "TSpawn", "MaxEHit", "X", "Y", "Z"]]),
        scaler.transform(testdata[["T", "E", "Size", "EToF", "EnergyMoment", "TSpawn", "MaxEHit", "X", "Y", "Z"]]),
    )
    for sname, scaler in scalers
]

Run all model/scaler combinations, in parallel. Note that we use timeouts per task, as setting at timeout in joblib will throw everything.

In [8]:
def with_timeout(timeout):
    def decorator(decorated):
        @functools.wraps(decorated)
        def inner(*args, **kwargs):
            pool = multiprocessing.pool.ThreadPool(1)
            async_result = pool.apply_async(decorated, args, kwargs)
            try:
                return async_result.get(timeout)
            except multiprocessing.TimeoutError:
                return

        return inner

    return decorator

In [9]:
MEDIHEAD = 500000 # 0
SLOWHEAD = 50000 # 0
label = "prim"

y_train_fast = traindata[[label]].values.ravel()
y_train_medi = traindata.head(MEDIHEAD)[[label]].values.ravel()
y_train_slow = traindata.head(SLOWHEAD)[[label]].values.ravel()
y_test = testdata[[label]].values.ravel()


@with_timeout(1200)
def train_model(mname, modelorg, speed, sname, x_train, x_test):
    # These get killed without error?
    if mname == "RadiusNeighborsClassifier" and sname != "Unscaled data":
        return (mname, sname, np.NaN, speed, np.NaN, "Skipped")
    # Too slow
    if mname == "LinearSVC" and (sname == "Unscaled data" or sname == "robust scaling"):
        return (mname, sname, np.NaN, speed, np.NaN, "Skipped")
    try:
        model = sklearn.base.clone(modelorg)
        start = time.time()
        if speed == "slow":
            model.fit(x_train[0:SLOWHEAD], y_train_slow)
        elif speed == "medi":
            model.fit(x_train[0:MEDIHEAD], y_train_medi)
        elif speed == "fast":
            model.fit(x_train, y_train_fast)
        end = time.time()

        y_pred = model.predict(x_test)
        y_true = y_test

        bac = sklearn.metrics.balanced_accuracy_score(y_true, y_pred)
        return (mname, sname, bac, speed, (end - start), "ok")
    except Exception as err:
        return (mname, sname, np.NaN, speed, np.NaN, err)


def train_model_wrap(mname, modelorg, speed, sname, x_train, x_test):
    ret = train_model(mname, modelorg, speed, sname, x_train, x_test)
    if ret:
        return ret
    else:
        return (mname, sname, np.NaN, speed, np.NaN, "Timeout")

In [10]:
try:
    results_a1 = Parallel(n_jobs=-1, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in trainscaled
        for mname, modelorg, speed in models_a1
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  40 | elapsed:    5.5s remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 11.4min finished


In [11]:
try:
    results_a2 = Parallel(n_jobs=-20, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in trainscaled
        for mname, modelorg, speed in models_a2
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=-20)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=-20)]: Done  48 out of  48 | elapsed:  7.1min finished


In [12]:
try:
    results_b = Parallel(n_jobs=-1, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in trainscaled
        for mname, modelorg, speed in models_b
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  56 | elapsed:   16.5s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  56 out of  56 | elapsed:  5.9min finished


In [13]:
try:
    results_s = Parallel(n_jobs=1, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in trainscaled
        for mname, modelorg, speed in models_s
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 33.2min finished


In [14]:
try:
    results_o = Parallel(n_jobs=-1, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in trainscaled
        for mname, modelorg, speed in models_o
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  24 | elapsed:  1.9min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  8.5min finished


In [15]:
def nullhypo():
    y_true = y_test
    y_pred = np.zeros(y_true.shape)

    bac = sklearn.metrics.balanced_accuracy_score(y_true, y_pred)
    return ("Null Hypothesis", "Unscaled data", bac, "fast", 0, "ok")

In [16]:
results = results_a1 + results_a2 + results_b + results_s + results_o + [nullhypo()]
resultsdf = pd.DataFrame(results)
pd.options.display.max_rows = 999
resultsdf.columns = ["Model", "Scaler", "BAC", "Speed", "Time", "Status"]
resultsdf.sort_values(by=["BAC", "Time", "Model"], ascending=[False, True, True], inplace=True)
#resultsdf.sort_values(by=["Model", "Scaler"], ascending=[True, True], inplace=True)
resultsdf.style.hide_index().format({"BAC": "{:.2%}", "Time": "{:.2f}"}).bar(subset=["BAC"], color="lightgreen").bar(
    subset=["Time"], color="lightblue"
)

Model,Scaler,BAC,Speed,Time,Status
RandomForestClassifier,quantile transformation (uniform pdf),90.25%,fast,90.05,ok
RandomForestClassifier,sample-wise L2 normalizing,90.24%,fast,106.81,ok
RandomForestClassifier,max-abs scaling,90.23%,fast,83.21,ok
RandomForestClassifier,min-max scaling,90.22%,fast,84.61,ok
RandomForestClassifier,Unscaled data,90.22%,fast,86.69,ok
RandomForestClassifier,robust scaling,90.22%,fast,89.00,ok
RandomForestClassifier,quantile transformation (gaussian pdf),90.21%,fast,88.77,ok
RandomForestClassifier,standard scaling,90.21%,fast,84.97,ok
ExtraTreesClassifier,sample-wise L2 normalizing,90.13%,fast,39.48,ok
ExtraTreesClassifier,standard scaling,90.09%,fast,35.26,ok
